In [14]:
import pandas as pd
import recordlinkage
import numpy as np
import timeit
from rapidfuzz import process

In [15]:
agg_debt = pd.read_csv('data/agg_debt_grouped.csv')
agg_debt.head()

C:\Users\david\AppData\Local\Temp\ipykernel_20260\786871525.py:1: DtypeWarning: Columns (2,10,11,13,15,16,20,21,22,24,31) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_debt = pd.read_csv('data/agg_debt_grouped.csv')


,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,C,8.0,27.0,1783.0,Elizabeth,Lowell,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,G,8.0,27.0,1783.0,Joshua,Brackett,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,C,9.0,2.0,1783.0,Phillips,White,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,B,9.0,2.0,1783.0,William,White,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
agg_debt.loc[agg_debt['org_file'] == 'Pierce_Certs_cleaned_2019.xlsx'].head()

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
14015,14015,15296.0,NaN,NaN,NaN,NaN,Mason,Abbe,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14016,14016,15298.0,NaN,NaN,NaN,NaN,John,Abbee,Jun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14017,14017,15299.0,NaN,NaN,NaN,NaN,Richard,Abbet,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14018,14018,15300.0,NaN,NaN,NaN,NaN,Reuben,Abbett,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14019,14019,15301.0,NaN,NaN,NaN,NaN,Richard,Abbett,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Create a full name column because process.extract can only compare one string at a time
agg_debt['full name'] = [' '.join([str(fn), str(ln)]) for fn, ln in zip(agg_debt['to whom due | first name'], agg_debt['to whom due | last name'])]
agg_debt['full name'].head()

0             nan nan
1    Elizabeth Lowell
2     Joshua Brackett
3      Phillips White
4       William White
Name: full name, dtype: object

In [18]:
# Remove 'cs' (congress) and 'f' (foreign officers); these are not states, but specific regiments / types of officers
agg_debt_copy = agg_debt[(agg_debt['state'] != 'cs') & (agg_debt['state'] != 'f') & (agg_debt['state'] != 'de')]

# Split dataframe based on state; makes searching faster
agg_debt_sp = agg_debt_copy.groupby('state')
agg_debts_st = [agg_debt_sp.get_group(x) for x in agg_debt_sp.groups]


for st in agg_debt_sp.groups:
    print(st)

ct
ga
ma
md
nc
nh
nj
ny
pa
ri
va


In [19]:
# Remove matches that have a score of 100 
def remove_exact_matches(matches):
    n_matches = []
    for match in matches:
        if match[1] < 100:
            n_matches.append(match)
    return n_matches

In [20]:
# Remove duplicate matches
def remove_dup_matches(matches):
    visited = {}
    output = []
    for name, score, index in matches:
        # Check if the first value is already present in the dictionary
        if name not in visited:
            visited[name] = True
            output.append((name, score, index))
    return output

In [21]:
%%timeit
for state in agg_debt_sp.groups:
    agg_debt_st = agg_debt_sp.get_group(state)
    # Iterate through each state's dataframe and .extract all the matches for each row 
    agg_debt_st['matches'] = agg_debt_st['full name'].apply(lambda full_name: process.extract(full_name, agg_debt_st['full name'], score_cutoff=90))
    print(state, str(len(agg_debt_st['matches'])))
    # Remove matches that have a score of 100 
    agg_debt_st['matches'] = agg_debt_st['matches'].apply(lambda matches: remove_exact_matches(matches))
    # Remove duplicate matches
    agg_debt_st['matches'] = agg_debt_st['matches'].apply(lambda matches: remove_dup_matches(matches)) 
    # Remove rows that have no matches 
    agg_debt_st_clean = agg_debt_st.loc[agg_debt_st.matches.str.len() > 0]
    # Save as a .csv file
    agg_debt_st_clean.to_csv('data/similar names/similar_names_' + state + '.csv') 

<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

ct 8436
ga 41


<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ma 16953


<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


va 1771


<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<magic-timeit>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

ct 8436
ga 41


KeyboardInterrupt: 